In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
total_data = pd.read_csv('uber_resource_with_weather.csv')

total_data = total_data[total_data['distance'] > 0.1]


In [ ]:
total_data.isnull().sum()

In [ ]:
label_weather = LabelEncoder()
total_data['weather'] = label_weather.fit_transform(total_data['short_summary'])
list(label_weather.classes_)

In [ ]:
label_location = LabelEncoder()
total_data['from'] = label_location.fit_transform(total_data['source'])
total_data['to'] = label_location.fit_transform(total_data['destination'])
list(label_location.classes_)

In [ ]:
label_car = LabelEncoder()
total_data['car'] = label_car.fit_transform(total_data['name'])
list(label_car.classes_)



In [ ]:
new_data = total_data[['weather', 'from', 'to', 'hour', 'day', 'month', 'car', 'price', 'distance']]

X = new_data.drop('price', axis=1)
y = new_data['price']
y = y.astype('int8')


In [ ]:
data = total_data.copy()
data['travel'] = data['source'] + '_'  + data['destination']
label_travel = LabelEncoder()
data['trip'] = label_travel.fit_transform(data['travel'])
data['unit'] = data['price'] / data['distance']
visual_data = data[['trip', 'hour', 'car', 'unit']]
list(label_travel.classes_)

In [ ]:
visual_data = visual_data.sort_values(by=['trip','hour'])
car_colors =  ['black', 'purple', 'red', 'orange', 'blue', 'green']
for trip in visual_data['trip'].unique():
    current_data = visual_data[visual_data['trip'] == trip]
    trip_name = label_travel.inverse_transform([trip])[0]
    graph_name = trip_name + ".png"
    plt.figure(figsize=(18, 18))
    plt.ylabel('unit price')
    plt.xlabel(trip_name)
    for line in current_data.iterrows():
        plt.plot(line[1]['hour'], line[1]['unit'], marker='x', color= car_colors[int(line[1]['car'])] )
    plt.savefig(graph_name)
    plt.clf()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.0005, random_state =42)

In [ ]:
sc =StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
mlpc = MLPClassifier(hidden_layer_sizes=(9,9,9),max_iter=150)
mlpc.fit(X_train, y_train)


In [ ]:
pred_mlpc = mlpc.predict(X_test)

In [ ]:
y_test.to_numpy()

In [ ]:
plt.figure(figsize=(18, 6))
plt.plot(y_test.to_numpy(), label="real",color='red')
plt.plot(pred_mlpc, label='predict', color='blue')
plt.ylabel('price')
plt.text(0,40,"Reality", color = 'red',)
plt.text(0,35,"Predict", color = 'Blue')
plt.savefig('price_est.png')

plt.show()
